In [ ]:
import numpy as np
import os
import pickle
import rasterio
import xarray as xr
from rasterio.transform import from_origin
from pathlib import Path
import joblib
import warnings

warnings.filterwarnings("ignore")

In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS
import matplotlib.pyplot as plt

folder = Path('EUMET')

# Get all file paths in the folder
eumet_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith("wmsMap")]
print(len(eumet_files))
eumet_files.sort()

# Define custom Mercator projection using PROJ string
mercator_crs = CRS.from_string(
    "+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=75 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"
)

for eumet_filepath in eumet_files:
    print(f"Opened: {eumet_filepath}")

    filename = os.path.basename(eumet_filepath)
    output_file = f'Reprojected_EUMET/{filename}'

    with rasterio.open(eumet_filepath) as src:
        # Read metadata and data
        data = src.read(2)  # Band indexing starts from 1 in rasterio

        # Access the profile (metadata) of the file
        profile = src.profile

        # Define the destination CRS
        dst_crs = mercator_crs.to_proj4()

        # Calculate the transform and size for the destination CRS
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        # Update the profile for the output file
        profile.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'dtype': 'float32'
        })

        # Reproject the data and write it to the new file
        with rasterio.open(output_file, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

        print(f"Reprojection complete. Output saved to {output_file}")

240
Opened: EUMET/wmsMap_2024-07-09T1845.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T1845.tif
Opened: EUMET/wmsMap_2024-07-09T1915.tif


/Users/pallavsaxena/miniconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:1293: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T1915.tif
Opened: EUMET/wmsMap_2024-07-09T1945.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T1945.tif
Opened: EUMET/wmsMap_2024-07-09T2015.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2015.tif
Opened: EUMET/wmsMap_2024-07-09T2045.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2045.tif
Opened: EUMET/wmsMap_2024-07-09T2115.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2115.tif
Opened: EUMET/wmsMap_2024-07-09T2145.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2145.tif
Opened: EUMET/wmsMap_2024-07-09T2215.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2215.tif
Opened: EUMET/wmsMap_2024-07-09T2245.tif
Reprojection complete. Output saved to Reprojected_EUMET/wmsMap_2024-07-09T2245.tif
Opened: EUMET/wmsMap_2024-07-09T2315.tif


In [4]:
import numpy as np
import rasterio
from scipy.interpolate import griddata
from pathlib import Path

interpolated_eumet_data_list = []

# Function to compute the coordinates from the affine transform
def compute_coordinates(transform, width, height):
    x_coords, y_coords = np.meshgrid(
        np.arange(width) * transform[0] + transform[2],
        np.arange(height) * transform[4] + transform[5]
    )
    return x_coords, y_coords

# Path to the folder containing the eumet files
folder = Path('Reprojected_EUMET')

# Get all file paths in the folder
eumet_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith("wmsMap")]
eumet_files.sort()

# Load and compute the coordinates for the INSAT dataset (common to all eumet files)
insat_filepath = 'INSAT/3RIMG_10JUL2024_0015_L1C_SGP_V01R00_IMG_TIR1.tif'
with rasterio.open(insat_filepath) as insat_src:
    insat_transform = insat_src.transform
    insat_width = insat_src.width
    insat_height = insat_src.height
    insat_x, insat_y = compute_coordinates(insat_transform, insat_width, insat_height)

# Prepare the INSAT grid where eumet data will be interpolated
insat_points = np.column_stack((insat_x.flatten(), insat_y.flatten()))

# Process each eumet file
for idx, eumet_filepath in enumerate(eumet_files):
    print(f"Opened: {eumet_filepath}")

    with rasterio.open(eumet_filepath) as eumet_src:
        # Read the second band (band 2) of the eumet file
        eumet_data = eumet_src.read(2)  # This is now a 2D array (height, width)
        eumet_transform = eumet_src.transform
        eumet_width = eumet_src.width
        eumet_height = eumet_src.height
        eumet_x, eumet_y = compute_coordinates(eumet_transform, eumet_width, eumet_height)

    # Flatten the eumet coordinates and data for griddata interpolation
    eumet_points = np.column_stack((eumet_x.flatten(), eumet_y.flatten()))
    eumet_values = eumet_data.flatten()  # Flatten the eumet data for interpolation

    # Interpolate the eumet data onto the INSAT grid
    interpolated_band = griddata(eumet_points, eumet_values, insat_points, method='linear')

    # Handle potential NaN values after interpolation
    interpolated_band = np.nan_to_num(interpolated_band)

    # Reshape the interpolated band to match the INSAT grid shape
    interpolated_data = interpolated_band.reshape(insat_x.shape)

    # Append the interpolated data for this eumet file
    interpolated_eumet_data_list.append(interpolated_data)
    print(f"Interpolation complete for file {idx + 1}. Shape of interpolated data: {interpolated_data.shape}")

Opened: Reprojected_EUMET/wmsMap_2024-07-09T1845.tif
Interpolation complete for file 1. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T1915.tif
Interpolation complete for file 2. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T1945.tif
Interpolation complete for file 3. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T2015.tif
Interpolation complete for file 4. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T2045.tif
Interpolation complete for file 5. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T2115.tif
Interpolation complete for file 6. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T2145.tif
Interpolation complete for file 7. Shape of interpolated data: (3207, 3062)
Opened: Reprojected_EUMET/wmsMap_2024-07-09T2215.tif
Interpolation complete for file 8. Shape of 

In [5]:
# save the interpolated data as joblib file
import joblib

joblib.dump(interpolated_eumet_data_list, 'interpolated_eumet_data_list.joblib')

['interpolated_eumet_data_list.joblib']

In [6]:
interpolated_eumet_data_list = joblib.load('interpolated_eumet_data_list.joblib')

In [7]:
import numpy as np
from collections import Counter

folder = Path('EUMET')
# Get all file paths in the folder
eumet_files = [file for file in folder.iterdir() if file.is_file() and file.name.startswith("wmsMap")]
print(len(eumet_files))
eumet_files.sort()

cloudfiltered_list = []

# Convert 0-255 values to cloud top heights (in meters)
min_height = 0     # Minimum cloud top height (0 km)
max_height = 15000 # Maximum cloud top height (15 km)

for i in range(len(eumet_files)):
    interpolated_eumet = interpolated_eumet_data_list[i]

    cth_in_meters = (interpolated_eumet / 255) * max_height

    cloudfiltered_mask = np.where((cth_in_meters>=1000) & (cth_in_meters<=9000), cth_in_meters, np.nan)     # all other cases → 1
    cloudfiltered_list.append(cloudfiltered_mask)                                                       

# Print the shape of eligibleCI filter of a particular file  and the counts
print(len(cloudfiltered_list))

240
240


In [8]:
# save the cloudfiltered_list in a joblib file
joblib.dump(cloudfiltered_list, "cloudfiltered_list.joblib")

['cloudfiltered_list.joblib']